<a href="https://colab.research.google.com/github/CMccormick0003/deep-learning-challenge/blob/main/AlphabetSoupCharity_optimized_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
print(application_df.shape)
print(application_df.columns)
application_df.head()

(34299, 12)
Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Preprocessing

In [124]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS'], inplace=True)

In [125]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [126]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df['APPLICATION_TYPE'].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [127]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(app_type_counts[app_type_counts < 51].index)
print(application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

['T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [128]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [129]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts_greater_than_one = classification_counts[classification_counts > 1]
print(classification_counts_greater_than_one)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [130]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 10].index)
print(classifications_to_replace)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

['C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: CLASSIFICATION, dtype: int64

In [131]:
# Look at ASK_AMT value counts for binning
ask_amt_counts = application_df['ASK_AMT'].value_counts()
print(ask_amt_counts)

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


In [132]:
# You may find it helpful to look at ASK_AMT value counts >2
ask_amt_counts = application_df['ASK_AMT'].value_counts()
ask_amt_counts_greater_than_two = ask_amt_counts[ask_amt_counts > 2]
print(ask_amt_counts_greater_than_two)

# the most common loan ask amount is $5000

5000     25398
10478        3
15583        3
63981        3
6725         3
Name: ASK_AMT, dtype: int64


In [133]:
application_df = application_df[application_df['ASK_AMT'] == 5000]
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
print(unique_counts)

APPLICATION_TYPE    11
AFFILIATION          6
CLASSIFICATION      30
USE_CASE             5
ORGANIZATION         4
STATUS               2
INCOME_AMT           9
ASK_AMT              1
IS_SUCCESSFUL        2
dtype: int64


In [134]:
# Convert categorical data to numeric with `pd.get_dummies`
# create a list of columns to encode
cat_cols = ['APPLICATION_TYPE', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'AFFILIATION']

# use pd.get_dummies to encode categorical variables
encoded_df = pd.get_dummies(application_df, columns=cat_cols)
encoded_df.shape

(25398, 68)

In [135]:
# Split our preprocessed data into our features and target arrays
X = encoded_df.drop("IS_SUCCESSFUL", axis=1).values
y = encoded_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [136]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print (X_train_scaled.shape)


(19048, 67)


## Compile, Train and Evaluate the Model

In [137]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = Sequential()

# First hidden layer
nn.add(Dense(units=80, activation='relu', input_dim=len(X_train_scaled[0])))

# Add 6 additional hidden layers
for i in range(6):
    nn.add(Dense(units=50, activation='relu'))

# Last hidden layer
nn.add(Dense(units=30, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_73 (Dense)            (None, 80)                5440      
                                                                 
 dense_74 (Dense)            (None, 50)                4050      
                                                                 
 dense_75 (Dense)            (None, 50)                2550      
                                                                 
 dense_76 (Dense)            (None, 50)                2550      
                                                                 
 dense_77 (Dense)            (None, 50)                2550      
                                                                 
 dense_78 (Dense)            (None, 50)                2550      
                                                                 
 dense_79 (Dense)            (None, 50)               

In [138]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [139]:
# Train the model
from tensorflow.keras.optimizers import SGD

sgd = SGD(learning_rate=0.1)

nn.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])


# Define the checkpoint path and filename
checkpoint_path = "model_weights/checkpoint_weights.{epoch:02d}.h5"

# Create a callback that saves the model's weights every five epochs
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=5 * len(X_train_scaled),
    verbose=1)

# Train the model with checkpoint callback
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
596/596 [==============================] - 2s 2ms/step - loss: 0.5685 - accuracy: 0.7210
Epoch 2/100
596/596 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7473
Epoch 3/100
596/596 [==============================] - 1s 1ms/step - loss: 0.5297 - accuracy: 0.7486
Epoch 4/100
596/596 [==============================] - 1s 1ms/step - loss: 0.5272 - accuracy: 0.7487
Epoch 5/100
596/596 [==============================] - 1s 1ms/step - loss: 0.5266 - accuracy: 0.7497
Epoch 6/100
596/596 [==============================] - 1s 1ms/step - loss: 0.5246 - accuracy: 0.7496
Epoch 7/100
596/596 [==============================] - 1s 1ms/step - loss: 0.5232 - accuracy: 0.7513
Epoch 8/100
596/596 [==============================] - 1s 1ms/step - loss: 0.5236 - accuracy: 0.7499
Epoch 9/100
596/596 [==============================] - 1s 2ms/step - loss: 0.5224 - accuracy: 0.7508
Epoch 10/100
596/596 [==============================] - 1s 2ms/step - loss: 0.5221 - accura

In [140]:
# Evaluate the model using the test data
evaluation = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {evaluation[0]}, Accuracy: {evaluation[1]}")

199/199 - 0s - loss: 0.5392 - accuracy: 0.7479 - 316ms/epoch - 2ms/step
Loss: 0.539186418056488, Accuracy: 0.7478740215301514


In [141]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityOptimised_7.h5")

In [142]:
# See the path of the current working directory
import os
print(os.getcwd())

/content
